In [ ]:
from pathlib import Path

import Metashape

from result import Ok, Err, Result

from benthoscan.backends import metashape as backend
from benthoscan.utils.log import logger

In [ ]:
PATHS: dict[str, Path] = {
    "DOCUMENT_IN": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
    ),
    "DOCUMENT_OUT": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects_test/r23685bc_working_version_saved.psz"
    ),
    "CACHE": Path("/home/martin/dev/benthoscan/.cache/"),
}

result: Result[str, str] = backend.load_project(PATHS.get("DOCUMENT_IN"))
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

backend.log_internal_data()

In [ ]:
from pathlib import Path

import Metashape

from tqdm import tqdm

def export_rendered_range(chunk: Metashape.Chunk, directory: Path) -> None:
    """Export range maps rendered from a Metashape mesh."""
    
    if chunk.transform.scale is None:
        scale: float = 1.0
    else:
    	scale: float = chunk.transform.scale
	
    
    for camera in tqdm(chunk.cameras, desc="Exporting rendered range maps..."):
        if not camera.transform:
            continue

        output_path: Path = directory / f"{camera.label}.tiff"
            
        range_map: Metashape.Image = chunk.model.renderDepth(camera.transform, camera.sensor.calibration, add_alpha=False)
        range_map: Metashape.Image = range_map * scale
        range_map: Metashape.Image = range_map.convert(" ","F32")
        
        compression = Metashape.ImageCompression()
        compression.tiff_compression = Metashape.ImageCompression().TiffCompressionDeflate
        
        range_map.save(str(output_path), compression=compression)

### Export range maps from Metashape

In [ ]:
document: Metashape.Document = backend.context._backend_data.get("document")

target_labels: list[str] = [ "r23685bc_20100605_021022" ]
target_chunks: list[Metashape.Chunk] = [chunk for chunk in document.chunks if chunk.label in target_labels]


output_root: Path = Path("/data/kingston_snv_01/rendered_range_maps")

# Export rendered depth
for chunk in target_chunks:
    directory: Path = output_root / f"{chunk.label}"
    export_rendered_range(chunk, directory=directory)